In [10]:
#import dependencies
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px

In [5]:
#read in source csv's
tx_df = pd.read_csv("TX_DEA_data.csv")
tx_county_data = pd.read_csv("tx_census_data.csv")
#tx_county_data = pd.read_csv('tx_data_js.csv')

In [6]:
#rename county column
tx_df = tx_df.rename(columns={ "BUYER_COUNTY" : "County" })

#convert county names to match census data for merge
tx_df["County"] = tx_df["County"].str.title()
tx_df["County"] = tx_df["County"] + (" County")

In [7]:
tx_county_data.head(5)


,Unnamed: 0,County,fips,Population,Avg. Income,Poverty %,Construction Jobs,STEM Jobs
0,0,Anderson County,48001,58084.0,18623.0,0.064217,5760.0,532.0
1,1,Andrews County,48003,16126.0,29363.0,0.046075,3700.0,260.0
2,2,Angelina County,48005,87433.0,21521.0,0.081342,8822.0,1375.0
3,3,Aransas County,48007,23889.0,25906.0,0.092888,2832.0,573.0
4,4,Archer County,48009,8853.0,28250.0,0.045747,1180.0,142.0


In [8]:
#rename columns
tx_county_data = tx_county_data.rename(columns={ 0 : "County", 1 : "fips", 2:'Population', 3:'Avg. Income', 4:'Poverty %', 5:'Construction Jobs', 6:'STEM Jobs'})
tx_county_data["County"] = tx_county_data["County"].str.split(pat = ",", n=1, expand=True)

In [9]:
#merge data frames on County names
merge_df = tx_df.merge(tx_county_data, how='left', on='County')
merge_df = merge_df.dropna()
merge_df.head(3)

,REPORTER_NAME,BUYER_NAME,BUYER_ADDRESS1,BUYER_STATE,County,BUYER_ZIP,TRANSACTION_DATE,CALC_BASE_WT_IN_GM,DRUG_NAME,QUANTITY,DOSAGE_UNIT,dos_str,Unnamed: 0,fips,Population,Avg. Income,Poverty %,Construction Jobs,STEM Jobs
0,BELLCO DRUG CORP,LONGHORN DRUG CO,1711 S. HENDERSON BLVD.,TX,Gregg County,75662,1112006,1.5135,HYDROCODONE,1.0,500.0,5.0,91.0,48183.0,122736.0,24137.0,0.080066,13274.0,3336.0
1,BELLCO DRUG CORP,MALLEY'S PHARMACY,MALLEY JOE DAVID,TX,Jefferson County,77706,1042006,1.5135,HYDROCODONE,1.0,500.0,5.0,122.0,48245.0,252466.0,23563.0,0.085085,26594.0,6472.0
2,BELLCO DRUG CORP,MALLEY'S PHARMACY,MALLEY JOE DAVID,TX,Jefferson County,77706,11022006,2.4216,HYDROCODONE,4.0,400.0,10.0,122.0,48245.0,252466.0,23563.0,0.085085,26594.0,6472.0


In [11]:
#group to sum dosage by county
dosage = merge_df.groupby("County")["DOSAGE_UNIT"].sum()
merge_df['Per Capita Dose'] = merge_df['DOSAGE_UNIT']/merge_df['Population']


fips = merge_df.groupby("County")["fips"].first()

In [12]:
pp_dose = merge_df.groupby('County')['Per Capita Dose'].sum()
len(pp_dose)

233

In [20]:
#plot total dosage chloropleth
fig = ff.create_choropleth(fips=fips, values=dosage, scope = ["Texas"])
fig.update_layout(
    title_text = 'Total Gross Dosage by County')
fig.show()
plt.savefig('texas_gross')

<Figure size 432x288 with 0 Axes>

In [21]:
#plot per capita dosage chloropleth
fig2 = ff.create_choropleth(fips=fips, values=pp_dose, scope = ["Texas"])
fig2.update_layout(
    title_text = "Total Dosage Per Capita by County")
fig2.show()
plt.savefig('texas_per_cap')

<Figure size 432x288 with 0 Axes>

In [29]:
stem_per_cap=merge_df.groupby('County')['STEM Jobs'].first()/merge_df.groupby('County')['Population'].first()

In [31]:
fig3 = ff.create_choropleth(fips=fips, values=stem_per_cap, scope = ["Texas"])
fig3.update_layout(
    title_text = "STEM Jobs Capita by County")
fig3.show()

In [24]:
len(merge_df['STEM Jobs'])

11951922